<a href="https://colab.research.google.com/github/jdl20515/IEEE-MURED-CV-Model/blob/main/Investigating_the_Effectiveness_of_Convolutional_Neural_Networks_on_Retinal_Disease_Diagnosis_Using_the_MURED_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the model that was used in the paper *Investigating the Effectiveness of Convolutional Neural Networks on Retinal Disease Diagnosis Using the MURED Dataset*, published in the 2024 IEEE IATMSI conference.

The full paper can be found at IEEE Explore: https://ieeexplore.ieee.org/document/10502464.

Jose David Lomelin; An Tran; Saumik Das; Lakshmisaketh Alluri; Rushil Challa; Sahil Sanjeev Narula; Aaditiya Jaganathan

In [ ]:
!pip install -q tensorflow_addons
!pip install tensorflow-ranking
#pip install pyyaml h5py

import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_addons as tfa
import zipfile
from google.colab import drive
import keras.api._v2.keras as keras
import random

from tensorflow.keras import Model, Input, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2M, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.applications.resnet_v2  import ResNet50V2, preprocess_input
from keras.layers import Dense, Flatten, Concatenate
from tensorflow.python.client import device_lib



# Run If Vgg16 does not work
# !pip uninstall tensorflow
# !pip install tensorflow==2.9.1

# Run if tfr/tfad does not work
#!pip install tensorflow-ranking
#!pip install tensorflow_addons



In [ ]:
# Loads image in from the set image path
#from PIL import
# Mount and Set Dataset Path (MURED)
# Can vary from 520x520 to 3400x2800
drive.mount('/content/drive', force_remount=True)
INPUT = (384, 384, )

#Define Noise Function
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 6

    #Add noise only 20% of the time
    rand = random.random()
    if rand > .8:
      deviation = VARIABILITY*random.random()
      noise = np.random.normal(0, deviation, img.shape)
      img += noise
      np.clip(img, 0., 255.)

    return img

# Add Data Augmentations Dataset
train = ImageDataGenerator(rescale = 1/255, width_shift_range=[-10, 10], height_shift_range=[-10, 10], brightness_range=[0.8, 1.2], horizontal_flip = True, zoom_range=.1, samplewise_center=True, samplewise_std_normalization=True, zca_whitening=True, preprocessing_function=add_noise,)
val = ImageDataGenerator(rescale = 1/255, width_shift_range=[-10, 10], height_shift_range=[-10, 10], brightness_range=[0.8, 1.2], horizontal_flip = True, zoom_range=.1, samplewise_center=True, samplewise_std_normalization=True, zca_whitening=True, preprocessing_function=add_noise,)
sample = ImageDataGenerator(rescale = 1/255)


#Define Training and Validation Set
training_set = train.flow_from_directory("/content/drive/MyDrive/MURED/train",
                                         target_size = INPUT,
                                         batch_size = 32,
                                         class_mode = "categorical",
                                         )

validation_set = val.flow_from_directory("/content/drive/MyDrive/MURED/val",
                                         target_size = INPUT,
                                         batch_size = 32,
                                         class_mode = "categorical",

                                         )

#Fit Generators
from numpy import asarray
from PIL import Image
spath = Image.open("/content/drive/MyDrive/MURED/small_sample/sample/23.png")
spath = spath.resize((384, 384))
simage = asarray(spath)
simage = simage.astype('float32')


simage = np.expand_dims(simage, axis=0)
print(simage.shape)

#train.fit(simage)
#val.fit(simage)



In [ ]:
# Make Model

# Make a copy of this model, with original idea (run xception, then resize with conv layers to fit the original image size, then pass into rest of models. Compare to this model's efficiency/time)
# DONT FORGET TO FREEZE NON FC LAYERS
# DONT FORGET TO DO CHECKPOINTS

INPUT = (384, 384, 3)
inputlayer = Input(shape=INPUT)

# Initializing Pretrained Models and Setting Convolutions to the Same Size
xception_model = Xception(weights = "imagenet", include_top=False, input_tensor=inputlayer)

conv1 = xception_model.output
conv1 = Conv2D(2048, (1, 1), activation="relu", padding="valid", strides=1)(conv1)
#conv1 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="valid")(conv1)


resNet_model = ResNet50V2(weights = "imagenet", include_top=False, input_tensor=inputlayer)
# for layers in resNet_model.layers[:-5]:
#   layers.trainable = False
conv2 = resNet_model.output
conv2 = Conv2D(2048, (1, 1), activation="relu", padding="valid", strides=1)(conv2)
#conv2 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="valid")(conv2)



# Concatenating the Base Models' Convolutions
concat1 = tf.concat([conv1, conv2], 3)
#concat1 = Flatten()(concat1)


# Convolutional and Pooling Layers to Base Models' Convolutions

# Xception Model
conv1 = Conv2D(128, (3, 3), activation="relu", padding="same", strides=1)(conv1)
conv1 = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(conv1)
conv1 = BatchNormalization()(conv1)

conv1 = Conv2D(256, (3, 3), activation="relu", padding="same", strides=1)(conv1)
conv1 = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(conv1)
conv1 = BatchNormalization()(conv1)

conv1 = Conv2D(2048, (1, 1), activation="relu", padding="same", strides=1)(conv1)
conv1 = BatchNormalization()(conv1)

# ResNet Model
conv2 = Conv2D(128, (3, 3), activation="relu", padding="same", strides=1)(conv2)
conv2 = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(conv2)
conv2 = BatchNormalization()(conv2)

conv2 = Conv2D(256, (3, 3), activation="relu", padding="same", strides=1)(conv2)
conv2 = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(conv2)
conv2 = BatchNormalization()(conv2)

conv2 = Conv2D(2048, (1, 1), activation="relu", padding="same", strides=1)(conv2)
conv2 = BatchNormalization()(conv2)

# Concatenating the new Convolutions
concat2 = tf.concat([conv1, conv2], 3)
#concat2 = Flatten()(concat2)


concat3 = tf.concat([concat1, concat2], 3)


#concat3 = tf.reshape(xceptionConv, shape=[15, 15, 2048])
concat3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')(concat3)
print(concat3)
concat3 = Conv2D(4096, (1, 1), activation="relu", padding="valid", strides=1)(concat3)

print(concat3)
concat3 = Flatten()(concat3)
print(concat3)



fc1 = Dense(2056, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.05, l2=0.05))(concat3)
fc1 = Dropout(0.05)(fc1)
fc1 = BatchNormalization()(fc1)

fc2 = Dense(2056, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.05, l2=0.05))(fc1)
fc2 = Dropout(0.05)(fc2)
fc2 = BatchNormalization()(fc2)

fc3 = Dense(1024, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.05, l2=0.05))(fc2)
fc3 = Dropout(0.05)(fc3)
fc3 = BatchNormalization()(fc3)

fc4 = Dense(512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.05, l2=0.05))(fc3)
fc4 = Dropout(0.05)(fc4)
fc4 = BatchNormalization()(fc4)



preds = Dense(19, activation="softmax")(fc4)

model = Model(inputs = inputlayer, outputs = preds)


# Freeze Pretrained Models
for layers in model.layers[:-42]:
   layers.trainable = False


In [ ]:
#@title Default title text
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.compat.v1.Session(config=config)
# Train and run model

'''
Importance Level    Hyperparameters

First               Learning Rate Alpha

Second              preprocessing
                    mini-batch size
                    number of hidden units/layers

Third               Adam beta1, beta2 (Between 1-0, higher means faster learning)
                    Dropout and L1/L2 regulrization
'''
epochs = 10
optimizer = Adam(learning_rate=0.000000003,
                 beta_1=0.9,
                 beta_2=0.999,)
model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics = [
                                                                                   tf.keras.metrics.CategoricalAccuracy(),
                                                                                   tf.keras.metrics.AUC(),
                                                                                   tfr.keras.metrics.MeanAveragePrecisionMetric(),
                                                                                   tfa.metrics.F1Score(num_classes=19)])


# Last train and load weights
# checkpoint_path = "/content/drive/MyDrive/training_1/cp-0007.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)


# !ls /content/drive/MyDrive/training_1
# latest = tf.train.latest_checkpoint(checkpoint_dir)
# print(latest)
model.load_weights("/content/drive/MyDrive/training_6/cp.hdf5")

# New train/weights
filepath = "/content/drive/MyDrive/training_6/cp.hdf5"
#new_dir = os.path.dirname(new_path)

# Create a callback that saves the model's weights every epoch
#epoch_size = 128
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    verbose=1,
    monitor='val_Categorical_Accuracy',
    save_weights_only=True,
    save_best_only=True,
    mode='max',
    )

# Fit model
# model.fit(training_set,
#           validation_data = validation_set,
#           epochs = 1,
#           #epochs = epochs,
#           callbacks=[cp_callback]
#           )

#Evaluate model
model.evaluate(validation_set,

          #epochs = 1,
          #epochs = epochs,
          #callbacks=[cp_callback]
          )
print(model.metrics_names)
print(model.metrics)
